In [1]:
#Import Libraries
import polars as pl
from dotenv import load_dotenv,find_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as chromeoptions
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
import json
import os
import re
from multiprocessing import Pool

In [2]:
#Creating webdriver instance as well as options for fullscreen
options = chromeoptions()
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install()))

In [3]:
#retrieve api key from hidden .env file
load_dotenv(find_dotenv())
hidden_key = os.getenv("hidden_api_key")

In [83]:
#getting url through webdriver and open instance of global crypto data
#creating parameters to search
symbol="BTC"
history_limit="365"
url=f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={symbol}&tsym=USD&limit=365&api_key={api_key}"
driver.get(url)

In [84]:
#see contents of page
page_source = driver.page_source

In [85]:
#clear all html denotations
cleaned_json = re.sub(r'<[^>]*>', '', page_source)

In [86]:
#verify data collected
data = json.loads(cleaned_json)

In [87]:
#display collected data
pl.DataFrame(data["Data"]["Data"])

time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
i64,f64,f64,f64,f64,f64,f64,str,str
1679097600,27749.91,26655.22,27441.41,48843.14,1.3355e9,26973.46,"""direct""",""""""
1679184000,28441.41,26901.66,26973.46,48163.58,1.3379e9,28040.29,"""direct""",""""""
1679270400,28539.63,27228.03,28040.29,65834.79,1.8387e9,27807.05,"""direct""",""""""
1679356800,28494.87,27416.8,27807.05,45696.4,1.2822e9,28188.11,"""direct""",""""""
1679443200,28902.98,26676.69,28188.11,81267.64,2.2779e9,27316.18,"""direct""",""""""
1679529600,28807.82,27190.13,27316.18,63194.55,1.7692e9,28345.7,"""direct""",""""""
1679616000,28421.94,27054.76,28345.7,52231.16,1.4570e9,27493.01,"""direct""",""""""
1679702400,27819.55,27186.95,27493.01,22474.29,6.1810e8,27492.83,"""direct""",""""""
1679788800,28217.66,27448.83,27492.83,26394.91,7.3460e8,27995.16,"""direct""",""""""


In [ ]:
#for symbol in symbols:
    #url=f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={symbol}&tsym=USD&limit=365&api_key={api_key}"

In [16]:
def get_data(symbol,api_key):
    url=f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={symbol}&tsym=USD&limit=365&api_key={api_key}"
    response=requests.get(url)
    data=response.json()["Data"]["Data"]
    data_with_symbol = [{'symbol': symbol, **item} for item in data]
    return data_with_symbol

In [17]:
if __name__ == "__main__":
    symbols=["BTC", "ETH", "LTC"]
    api_key= hidden_key

In [18]:
alls_df =[]
with Pool(4) as pool:
    for symbol in symbols:
        result= get_data(symbol, api_key)
        alls_df.append(pl.DataFrame(result))

In [21]:
df_final=pl.concat(alls_df)

In [30]:
df_final

symbol,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
str,i64,f64,f64,f64,f64,f64,f64,str,str
"""BTC""",1679097600,27749.91,26655.22,27441.41,48843.14,1.3355e9,26973.46,"""direct""",""""""
"""BTC""",1679184000,28441.41,26901.66,26973.46,48163.58,1.3379e9,28040.29,"""direct""",""""""
"""BTC""",1679270400,28539.63,27228.03,28040.29,65834.79,1.8387e9,27807.05,"""direct""",""""""
"""BTC""",1679356800,28494.87,27416.8,27807.05,45696.4,1.2822e9,28188.11,"""direct""",""""""
"""BTC""",1679443200,28902.98,26676.69,28188.11,81267.64,2.2779e9,27316.18,"""direct""",""""""
"""BTC""",1679529600,28807.82,27190.13,27316.18,63194.55,1.7692e9,28345.7,"""direct""",""""""
"""BTC""",1679616000,28421.94,27054.76,28345.7,52231.16,1.4570e9,27493.01,"""direct""",""""""
"""BTC""",1679702400,27819.55,27186.95,27493.01,22474.29,6.1810e8,27492.83,"""direct""",""""""
"""BTC""",1679788800,28217.66,27448.83,27492.83,26394.91,7.3460e8,27995.16,"""direct""",""""""
